In [1]:
import numpy as np
all_selected_features = np.load('all_selected_features.npy', encoding = 'latin1')

In [2]:
tags = np.load('tags.npy', encoding = 'latin1')

In [3]:
all_selected_features

array([[4.0, 24.0, 10.491330295138889, ..., 0.04764792486751901,
        -0.20372739284305696, 0.007738569507839782],
       [4.18, 25.0, 7.1436875, ..., 0.05345539157154828,
        -0.09267610658280212, 0.5205620970512248],
       [3.12, 29.97002997002997, 18.564969618055557, ...,
        0.04755244724276521, 0.45511116341639024, 0.13341562334134907],
       ...,
       [3.68, 29.97002997002997, 50.58107638888889, ...,
        0.08872334474799293, 0.8049269913986392, 0.5518088927610451],
       [3.42, 29.97002997002997, 13.070269097222223, ...,
        0.04186467671244019, 1.011901183110088, 0.49866779137366257],
       [3.4, 29.97002997002997, 119.80957899305555, ...,
        0.050812777751733096, 0.8206719296310854, 0.6348684952344302]],
      dtype=object)

In [4]:
from sklearn.utils import shuffle
all_features, all_MOS = shuffle(all_selected_features[:,1:], all_selected_features[:,0]/5, random_state=1 )

In [5]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=all_features,label=all_MOS)


regr = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

#xg_reg.fit(all_features,all_MOS)

#preds = xg_reg.predict(X_test)

/home/hatef/.local/lib/python3.5/site-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [6]:
def MSE(seq1, seq2, length):
    sum_square_error = 0
    for i in range(length):
        sum_square_error += (seq1[i]-seq2[i])**2
        
    mean_square_error =sum_square_error/length
    return mean_square_error

import math
def RMSE(seq1, seq2, length):
    return math.sqrt(MSE(seq1, seq2, length))

def MAE(seq1, seq2, length):
    sum_abs_error = 0
    for i in range(length):
        sum_abs_error += abs(seq1[i]-seq2[i])
        
    mean_abs_error = sum_abs_error/length
    return mean_abs_error



from sklearn.model_selection import KFold
import scipy as scy
sum_spearmanr_train = sum_pearsonr_train = sum_kendalltau_train =  sum_MSE_train = sum_RMSE_train = sum_MAE_train = 0
sum_spearmanr_test  = sum_pearsonr_test  = sum_kendalltau_test  =  sum_MSE_test  = sum_RMSE_test  = sum_MAE_test  = 0
num_folds = 10
k=0
kf = KFold(n_splits=num_folds,random_state=1)
for train, test in kf.split(all_selected_features):
    
    all_features_train, all_MOS_train = [all_selected_features[train,1:], all_selected_features[train,0]/5.0]
    all_features_test,  all_MOS_test  = [all_selected_features[test,1:],  all_selected_features[test,0]/5.0 ]
    
    train_num = all_features_train.shape[0]
    test_num  = all_features_test.shape[0]
    
    list_features_train = []
    list_features_test  = []
    
    list_MOS_train = []
    list_MOS_test  = []
    
    for i in range(train_num):
        list_features_train.append(all_features_train[i,:])
        list_MOS_train.append(all_MOS_train[i])
    
    for i in range(test_num):
        list_features_test.append(all_features_test[i,:])
        list_MOS_test.append(all_MOS_test[i])
    
    # redifine regr
    regr = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
    regr = regr.fit(np.array(list_features_train), np.array(list_MOS_train))
    
    ##---------------------------------------
    k+=1
    print("------------- train ",k,"/",num_folds,"-------------")
    MOS_pred = regr.predict(np.array(list_features_train))
    
    spearmanr_train=scy.stats.spearmanr(list_MOS_train,MOS_pred)
    #print("Spearman Correlation result of train")
    print(spearmanr_train)
    
    pearsonr_train=scy.stats.pearsonr(list_MOS_train,MOS_pred)
    print("Pearson Correlation result of train")
    print(pearsonr_train)
    
    kendalltau_train=scy.stats.kendalltau(list_MOS_train,MOS_pred)
    #print("Kendall's  Correlation result of train")
    print(kendalltau_train)
    
    MSE_train=MSE(list_MOS_train,MOS_pred,len(list_MOS_train))
    print("MSE of train")
    print(MSE_train)
    
    RMSE_train=RMSE(list_MOS_train,MOS_pred,len(list_MOS_train))
    print("RMSE of train")
    print(RMSE_train)
    
    MAE_train=MAE(list_MOS_train,MOS_pred,len(list_MOS_train))
    print("MAE of train")
    print(MAE_train)
    
    
    ##---------------------------------------
    print("------------- test -------------")
    
    MOS_pred = regr.predict(np.array(list_features_test))
    
    spearmanr_test = scy.stats.spearmanr(list_MOS_test,MOS_pred)
    #print("Spearman Correlation result of test")
    print(spearmanr_test)
    
    pearsonr_test=scy.stats.pearsonr(list_MOS_test,MOS_pred)
    print("Pearson Correlation result of test")
    print(pearsonr_test)
    
    kendalltau_test=scy.stats.kendalltau(list_MOS_test,MOS_pred)
    #print("Kendall's  Correlation result of test")
    print(kendalltau_test)
    
    MSE_test=MSE(list_MOS_test,MOS_pred,len(list_MOS_test))
    print("MSE of test")
    print(MSE_test)
    
    RMSE_test=RMSE(list_MOS_test,MOS_pred,len(list_MOS_test))
    print("RMSE of test")
    print(RMSE_test)
    
    MAE_test=MAE(list_MOS_test,MOS_pred,len(list_MOS_test))
    print("MAE of test")
    print(MAE_test)
    
    

    sum_spearmanr_train += spearmanr_train[0]
    sum_pearsonr_train  += pearsonr_train[0]
    sum_kendalltau_train+= kendalltau_train[0]
    sum_MSE_train += MSE_train 
    sum_RMSE_train += RMSE_train 
    sum_MAE_train += MAE_train 
    
    sum_spearmanr_test += spearmanr_test[0]
    sum_pearsonr_test  += pearsonr_test[0]
    sum_kendalltau_test+= kendalltau_test[0]
    sum_MSE_test += MSE_test 
    sum_RMSE_test += RMSE_test 
    sum_MAE_test += MAE_test


[20:20:45] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:20:45] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
------------- train  1 / 10 -------------
SpearmanrResult(correlation=0.7543780852467492, pvalue=1.0206990666327191e-168)
Pearson Correlation result of train
(0.7532027235119575, 6.627675628003569e-168)
KendalltauResult(correlation=0.573470369043416, pvalue=1.2509588422965645e-136)
MSE of train
0.012027481302978933
RMSE of train
0.10966987418146759
MAE of train
0.09172290332432775
------------- test -------------
SpearmanrResult(correlation=0.6540468973824581, pvalue=9.077670468096291e-14)
Pearson Correlation result of test
(0.66117517534779, 3.9332196073735444e-14)
KendalltauResult(correlation=0.48942301699795715, pvalue=2.384818754070542e-12)
MSE of test
0.014583814275726758
RMSE of test
0.12076346415918499
MAE of test
0.0994845247197

In [7]:
sum_spearmanr_train = sum_spearmanr_train/float(num_folds)
sum_pearsonr_train  = sum_pearsonr_train/float(num_folds)  
sum_kendalltau_train= sum_kendalltau_train/float(num_folds)
sum_MSE_train = sum_MSE_train /float(num_folds)
sum_RMSE_train = sum_RMSE_train /float(num_folds)
sum_MAE_train = sum_MAE_train /float(num_folds)

sum_spearmanr_test = sum_spearmanr_test/float(num_folds)
sum_pearsonr_test  = sum_pearsonr_test/float(num_folds)  
sum_kendalltau_test= sum_kendalltau_test/float(num_folds)
sum_MSE_test = sum_MSE_test /float(num_folds)
sum_RMSE_test = sum_RMSE_test /float(num_folds)
sum_MAE_test = sum_MAE_test /float(num_folds)

print("------------- train -------------")
print("Spearman Correlation result of train")
print(sum_spearmanr_train)

print("Pearson Correlation result of train")
print(sum_pearsonr_train)

print("Kendall's  Correlation result of train")
print(sum_kendalltau_train)

print("MSE of train")
print(sum_MSE_train)

print("RMSE of train")
print(sum_RMSE_train)

print("MAE of train")
print(sum_MAE_train)

#####################################
print("------------- test -------------")
print("Spearman Correlation result of test")
print(sum_spearmanr_test)

print("Pearson Correlation result of test")
print(sum_pearsonr_test)

print("Kendall's  Correlation result of test")
print(sum_kendalltau_test)

print("MSE of test")
print(sum_MSE_test)

print("RMSE of test")
print(sum_RMSE_test)

print("MAE of test")
print(sum_MAE_test)



# Plot
import matplotlib.pyplot as plt
MOS_pred = regr.predict(np.array(list_features_train))

plt.scatter(list_MOS_train,MOS_pred , alpha=0.5)
plt.title('Train data')
plt.xlabel('Ground-truth MOS')
plt.ylabel('Predcited MOS')
plt.savefig('SVR_train',dpi=200)
plt.show()


MOS_pred = regr.predict(np.array(list_features_test))

plt.scatter(list_MOS_test,MOS_pred , alpha=0.5)
plt.title('Test data')
plt.xlabel('Ground-truth MOS')
plt.ylabel('Predcited MOS')
plt.savefig('SVR_test',dpi=200)
plt.show() 

------------- train -------------
Spearman Correlation result of train
0.7534948065872433
Pearson Correlation result of train
0.752400145944536
Kendall's  Correlation result of train
0.5715159257572291
MSE of train
0.012123807326117458
RMSE of train
0.11010625040878222
MAE of train
0.0918652884064227
------------- test -------------
Spearman Correlation result of test
0.6713792805692724
Pearson Correlation result of test
0.6757681559463949
Kendall's  Correlation result of test
0.49670502023079105
MSE of test
0.013025355166563678
RMSE of test
0.11392412487983528
MAE of test
0.09520722677195934


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>